<a href="https://colab.research.google.com/github/SVz54/Data-Exfiltration-Prevention/blob/main/9517.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- Kaggle download (same as before) ---
import os, json, shutil, zipfile, glob, pathlib

os.makedirs('/root/.kaggle', exist_ok=True)
if not os.path.exists('/root/.kaggle/kaggle.json') and os.path.exists('/content/kaggle.json'):
    shutil.move('/content/kaggle.json', '/root/.kaggle/kaggle.json')
!chmod 600 /root/.kaggle/kaggle.json

!pip -q install kaggle
DATA_DIR = "/content/AgroPest12"
os.makedirs(DATA_DIR, exist_ok=True)
!kaggle datasets download -d rupankarmajumdar/crop-pests-dataset -p $DATA_DIR -q

# Unzip (overwrite if re-running)
zip_files = glob.glob(f"{DATA_DIR}/*.zip")
assert zip_files, "Zip not found – did the Kaggle download succeed?"
zip_path = zip_files[0]
!unzip -q -o "$zip_path" -d "$DATA_DIR"

# --- Auto-detect BASE: the folder that contains train/valid/test with images+labels ---
def find_yolo_base(root):
    for p, d, f in os.walk(root):
        if (os.path.isdir(os.path.join(p, "train", "images")) and
            os.path.isdir(os.path.join(p, "train", "labels")) and
            os.path.isdir(os.path.join(p, "valid", "images")) and
            os.path.isdir(os.path.join(p, "valid", "labels"))):
            return p
    return None

BASE = find_yolo_base(DATA_DIR)
assert BASE is not None, f"Could not find YOLO folders under {DATA_DIR}. Found: {os.listdir(DATA_DIR)}"
print("BASE:", BASE)
print("train samples:", len(glob.glob(os.path.join(BASE, "train/images/*.jpg"))))
print("val samples:", len(glob.glob(os.path.join(BASE, "valid/images/*.jpg"))))
print("test samples:", len(glob.glob(os.path.join(BASE, "test/images/*.jpg"))))
BASE = pathlib.Path(BASE)  # keep as Path for later cells


Dataset URL: https://www.kaggle.com/datasets/rupankarmajumdar/crop-pests-dataset
License(s): MIT
BASE: /content/AgroPest12
train samples: 11502
val samples: 1095
test samples: 546


In [2]:
from pathlib import Path
yaml_path = BASE / "data.yaml"
txt = f"""# AgroPest-12
path: {BASE.as_posix()}
train: train/images
val: valid/images
test: test/images
names:
  0: aphid
  1: armyworm
  2: beetle
  3: bollworm
  4: grasshopper
  5: leafhopper
  6: locust
  7: mealybug
  8: mosquito
  9: moth
  10: sawfly
  11: weevil
"""
yaml_path.write_text(txt)
print("Wrote:", yaml_path)


Wrote: /content/AgroPest12/data.yaml


In [3]:
from ultralytics import YOLO
import os, glob

model = YOLO('yolov8n.pt')   # tiny + fast; swap to yolov8s.pt later if you want

# use a handful of val images for a demo run
val_imgs = sorted(glob.glob(str(BASE / 'valid/images/*.jpg')))[:12]
print("Demo images:", len(val_imgs))

pred_root = "/content/preds"
res = model.predict(val_imgs, conf=0.25, save=True, project=pred_root, name="yolo_preds", exist_ok=True, imgsz=640)
print("Saved predicted images to:", pred_root + "/yolo_preds")


Demo images: 12

0: 640x640 (no detections), 339.1ms
1: 640x640 (no detections), 339.1ms
2: 640x640 1 bear, 339.1ms
3: 640x640 (no detections), 339.1ms
4: 640x640 1 cat, 339.1ms
5: 640x640 1 horse, 339.1ms
6: 640x640 (no detections), 339.1ms
7: 640x640 1 person, 339.1ms
8: 640x640 1 bird, 1 horse, 339.1ms
9: 640x640 1 teddy bear, 339.1ms
10: 640x640 1 bird, 339.1ms
11: 640x640 (no detections), 339.1ms
Speed: 12.0ms preprocess, 339.1ms inference, 3.2ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /content/preds/yolo_preds
Saved predicted images to: /content/preds/yolo_preds


In [4]:
!pip -q uninstall -y pytorch-grad-cam grad-cam || true
!pip -q install --no-cache-dir "git+https://github.com/jacobgil/pytorch-grad-cam.git"


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [5]:
import pytorch_grad_cam, inspect
from pytorch_grad_cam.utils import model_targets
print("grad-cam version:", getattr(pytorch_grad_cam, "__version__", "git"))
print("Has YOLOv8Target:", hasattr(model_targets, "YOLOv8Target"))


grad-cam version: git
Has YOLOv8Target: False


In [6]:
# --- Single-image, low-memory, true Grad-CAM on YOLOv8 (crop-only) ---
import os, gc, cv2, torch, numpy as np, torch.nn as nn
from ultralytics import YOLO
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

os.environ["OMP_NUM_THREADS"] = "1"

# 1) detection model (reuse your existing one)
model_det = YOLO('yolov8n.pt')    # or reuse 'model' if it's already loaded

# 2) clean model for CAM (never use .predict() on this)
model_cam = YOLO('yolov8n.pt')
device = "cpu"                     # keep CPU for stability
model_cam.model.to(device).eval()

def last_conv2d(ultra_model):
    last = None
    for _, m in ultra_model.model.named_modules():
        if isinstance(m, nn.Conv2d):
            last = m
    return last

target_layer = last_conv2d(model_cam)
assert target_layer is not None

def load_rgb_float(path):
    bgr = cv2.imread(path)
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    return rgb, (rgb.astype(np.float32)/255.0)

def clamp_int(v, lo, hi):
    return max(lo, min(int(v), hi))

# Pick the FIRST val image that yields any detection with the COCO model
chosen = None
chosen_det = None
for ip in sorted((BASE/'valid/images').glob('*.jpg'))[:50]:  # look at at most 50
    r = model_det(str(ip), conf=0.35, imgsz=320)[0]
    if r.boxes is not None and len(r.boxes) > 0:
        chosen = str(ip)
        chosen_det = r
        break

if chosen is None:
    raise RuntimeError("No detections found on first 50 val images with COCO weights; try lowering conf or pick another image.")

print("Using image:", chosen)
rgb, rgbf = load_rgb_float(chosen)
h, w = rgb.shape[:2]

# Take top-confidence detection and crop around it (shrinks tensor size a lot)
k = int(torch.argmax(chosen_det.boxes.conf).item())
x1,y1,x2,y2 = chosen_det.boxes.xyxy[k].tolist()
pad = 0.15
xa = clamp_int(x1 - (x2-x1)*pad, 0, w-1)
ya = clamp_int(y1 - (y2-y1)*pad, 0, h-1)
xb = clamp_int(x2 + (x2-x1)*pad, 0, w-1)
yb = clamp_int(y2 + (y2-y1)*pad, 0, h-1)

crop = rgb[ya:yb, xa:xb]
cropf = crop.astype(np.float32)/255.0
inp = torch.from_numpy(cropf.transpose(2,0,1)).unsqueeze(0).to(device).float()
inp.requires_grad_(True)

# Rerun detection on the CROP to get a matching class index in the crop frame
r_crop = model_det(crop[..., ::-1], conf=0.25, imgsz=320)[0]  # BGR expected; we pass RGB->BGR by reversing channels
if r_crop.boxes is None or len(r_crop.boxes)==0:
    raise RuntimeError("No detection in crop; try a different image or lower conf.")

top_cls = int(r_crop.boxes.cls[torch.argmax(r_crop.boxes.conf)].item())

# Custom class-specific target: pick the strongest prediction row for 'top_cls'
class YoloRowTarget:
    def __init__(self, cls_idx, model_ref):
        self.cls_idx = int(cls_idx)
        self.nc = model_ref.model.model[-1].nc
    def __call__(self, outputs):
        pred = outputs[0]  # (N, 4+nc[+obj])
        if pred.size(1) > 4 + self.nc:
            obj = pred[:,4].sigmoid()
            cls = pred[:,5 + self.cls_idx].sigmoid()
            score = (obj*cls).max()
        else:
            cls = pred[:,4 + self.cls_idx].sigmoid()
            score = cls.max()
        return score

target = YoloRowTarget(top_cls, model_cam)

cam = GradCAM(model=model_cam.model, target_layers=[target_layer])
cam_map = cam(input_tensor=inp, targets=[target], eigen_smooth=False)[0]  # no smoothing to save RAM

# Overlay and save
vis = show_cam_on_image(cropf, cam_map, use_rgb=True)
out_dir = "/content/gradcam"
os.makedirs(out_dir, exist_ok=True)
out = os.path.join(out_dir, os.path.basename(chosen).replace(".jpg","_CROP_gradcam.jpg"))
cv2.imwrite(out, cv2.cvtColor(vis, cv2.COLOR_RGB2BGR))
print("Saved:", out)

# cleanup
del inp, cam_map, crop, cropf, r_crop, chosen_det
gc.collect()



image 1/1 /content/AgroPest12/valid/images/Weevil-101-_jpg.rf.7b2714887709397bf4467aee16ea2e79.jpg: 320x320 1 bird, 109.7ms
Speed: 8.8ms preprocess, 109.7ms inference, 1.2ms postprocess per image at shape (1, 3, 320, 320)
Using image: /content/AgroPest12/valid/images/Weevil-101-_jpg.rf.7b2714887709397bf4467aee16ea2e79.jpg

0: 320x320 1 bird, 99.1ms
Speed: 6.9ms preprocess, 99.1ms inference, 1.4ms postprocess per image at shape (1, 3, 320, 320)
Saved: /content/gradcam/Weevil-101-_jpg.rf.7b2714887709397bf4467aee16ea2e79_CROP_gradcam.jpg


10